# Policy RNN for Part-of-Speech Tagging

In [2]:
# Load Part-of-Speech data
from lxmls.readers.pos_corpus import PostagCorpusData
data = PostagCorpusData()
# Get batch iterators for train and test
batch_size = 1
train_batches = data.batches('train', batch_size=batch_size)
dev_set = data.batches('dev', batch_size=batch_size)
test_set = data.batches('test', batch_size=batch_size)

In [4]:
reinforce = True
# Load version of the RNN
from lxmls.deep_learning.pytorch_models.rnn import PolicyRNN
model = PolicyRNN(
input_size=data.input_size,
embedding_size=50,
hidden_size=100,
output_size=data.output_size,
learning_rate=0.05,
gamma=0.8,
RL=reinforce,
maxL=data.maxL
)

In [5]:
import numpy as np
import time

# Run training
num_epochs = 15
batch_size = 1

# Get batch iterators for train and test
train_batches = data.sample('train', batch_size=batch_size)
dev_set = data.sample('dev', batch_size=batch_size)
test_set = data.sample('test', batch_size=batch_size)

# Epoch loop
start = time.time()
for epoch in range(num_epochs):
    # Batch loop
    for batch in train_batches:
        model.update(input=batch['input'], output=batch['output'])
    # Evaluation dev
    is_hit = []
    for batch in dev_set:
        loss = model.predict_loss(batch['input'], batch['output'])
        is_hit.extend(loss)
    accuracy = 100 * np.mean(is_hit)
    # Inform user
    print("Epoch %d: dev accuracy %2.2f %%" % (epoch + 1, accuracy))
    
print("Training took %2.2f seconds per epoch" % ((time.time() - start) / num_epochs))
# Evaluation test
is_hit = []
for batch in test_set:
    is_hit.extend(model.predict_loss(batch['input'],batch['output']))
accuracy = 100 * np.mean(is_hit)

# Inform user
print("Test accuracy %2.2f %%" % accuracy)

Epoch 1: dev accuracy 79.95 %
Epoch 2: dev accuracy 85.14 %
Epoch 3: dev accuracy 85.20 %
Epoch 4: dev accuracy 85.20 %
Epoch 5: dev accuracy 85.20 %
Epoch 6: dev accuracy 85.20 %
Epoch 7: dev accuracy 85.23 %
Epoch 8: dev accuracy 85.20 %
Epoch 9: dev accuracy 85.23 %
Epoch 10: dev accuracy 85.20 %
Epoch 11: dev accuracy 85.23 %
Epoch 12: dev accuracy 85.20 %
Epoch 13: dev accuracy 85.23 %
Epoch 14: dev accuracy 85.20 %
Epoch 15: dev accuracy 85.23 %
Training took 3.39 seconds per epoch
Test accuracy 84.11 %
